In [13]:
from aima import csp
from aima import search
from aima import agents
from itertools import product, groupby
from operator import itemgetter
import random
import re


In [57]:
def make_vars(nscrambles, nforms, nblocks, master=True):
    blocks = range(1, nblocks + 1)
    forms = range(1, nforms + 1)
    out_dict = {(0, i) : [0] for i in blocks}
    scrambles = range(1 if not master else 0, nscrambles + 1)
    out_dict.update({tup : scrambles[:] for tup in product(forms, blocks)})
    return out_dict


def make_neighbors(variables):
    key_func = itemgetter(0)
    sorted_by_block = sorted(variables.keys(), key=key_func)
    grped = groupby(sorted_by_block, key=key_func)
    out_dict = {}
    for _, group in grped:
        group = sorted(group)
        out_dict[group[0]] = [group[1]]
        out_dict[group[-1]] = [group[-2]]
        for i, var in enumerate(group[1:-1], start=1):
            out_dict[var] = [group[i-1], group[i+1]]
    return out_dict
            

def extract_args_as_attr(**kwds):
    def decorator(f):
        argspec = inspect.getargspec(f)
        def wrapper(*args, **kwargs):
            self = args[0]
            for var, attr in kwds.items():
                if var in kwargs:
                    setattr(self, attr, kwargs[var])
                else:
                    setattr(self, attr, args[argspec.args.index(var)])
            return f(*args, **kwargs)
        return wrapper
    return decorator

assign_extractor = extract_args_as_attr(assignment='_assignment')

class FormScramblingCSP(csp.CSP):
    
    def __init__(self, variables, domains, neighbors, key_dict, global_constraints):
        csp.CSP.__init__(self, variables, domains, neighbors, self.constraints)
        self._global_constraints = global_constraints
        self._key_dict = key_dict

    def _check_neighbors(self, A, a, B, b):
        
        a_key = self._key_dict[A[1]][a]
        b_key = self._key_dict[B[1]][b]
        if A[1] < B[1]:
            key = a_key + b_key
            return not any(all(chnk[0] == c for c in chnk[1:]) for chnk in iter_chunks())
    
    def _check_friends(self, A, a, b, B):
        return True
    
    def _check_enemies(self, A, a, B, b):
        return True
    
    def _check_block_overlaps(self, A, a, B, b):
        return True
    
    def _check_key_overlaps(self, A, a, B, b):
        return True
    
    def constraints(self, A, a, B, b):
#         print(self._assignment)
        print(B in self.neighbors[A])
        return True
    
    unassign = assign_extractor(csp.CSP.unassign)
    nconflicts = assign_extractor(csp.CSP.nconflicts)
    assign = assign_extractor(csp.CSP.assign)
    display = assign_extractor(csp.CSP.display)
    


In [53]:
import inspect

def extract_args_as_attr(**kwds):
    def decorator(f):
        argspec = inspect.getargspec(f)
        def wrapper(*args, **kwargs):
            self = args[0]
            for var, attr in kwds.items():
                if var in kwargs:
                    setattr(self, attr, kwargs[var])
                else:
                    setattr(self, attr, args[argspec.args.index(var)])
            return f(*args, **kwargs)
        return wrapper
    return decorator

class MyClass(object):    
    @extract_args_as_attr(a='_a', b='_b')
    def my_method(self, a, b):
        return a + b
    

c = MyClass()
    

In [58]:
variables = make_vars(7, 5, 9)
neighbors = make_neighbors(variables)

my_csp = FormScramblingCSP(variables.keys(), variables, neighbors, {})

In [20]:
public_attrs = [attr for attr in csp.CSP.__dict__ if callable(csp.CSP.__dict__[attr]) and 'assignment' in inspect.getargspec(csp.CSP.__dict__[attr]).args]
public_attrs

['unassign', 'nconflicts', 'assign', 'display']

In [61]:
from itertools import islice, combinations
from collections import defaultdict, Counter

def all_equal(a):
    a_it = iter(a)
    a_0 = next(a_it)
    return all(a_0 == a_i for a_i in a_it)


def iter_chunks(a, chunk_size):
    it_a = iter(a)
    while True:
        out = list(islice(it_a, chunk_size))
        if out: 
            yield out
        else: 
            break

            
def count_overlaps(a, b):
    return sum(a_i == b_i for a_i, b_i in zip(a, b))

class GlobalConstraints(object):
    
    def __init__(self, nforms, key_dict, variables, key_overlaps=24, n_border=3, use_all=True, max_reuse=2):
        self._nforms = nforms
        self._key_dict = key_dict
        self._variables = sorted(variables)
        self._key_overlaps = key_overlaps
        self._n_border = n_border
        self._use_all = use_all
        self._max_reuse = max_reuse 
        
    def __call__(self, A, a, B, b, assignment):
        assignment = assignment.copy()
        assignment[A] = a
        assignment[B] = b
        return all(
            self._check_borders(A, a, B, b),
            self._check_usage(assignment),
            self._check_key_overlaps(assignment)
        )
    
    def _check_borders(self, A, a, B, b):
        assert A[0] == B[0]
        if A[1] < B[1]:
            first, second = (A[1], a), (B[1], b)
        else:
            first, second = (B[1], b), (A[1], a)
        border_seq = self._key_dict[first] + self._key_dict[second]
        return not any(all_equal(chunk) for chunk in iter_chunks(border_seq, self._n_border))
    
    
    def _get_form_key_dict(self, assignment):
        form_key_dict = defaultdict(list)
        #key_dict is {(block, scramble) : answer_key}
        #assignment is {(form, block) : scramble}
        for var in self._variables:
            if var in assignment:
                form_key_dict[var[0]].append(self._key_dict[(var[1], assignment[var])])
            else:
                form_key_dict[var[0]].append([])
        return form_key_dict
    
    
    def _check_overlaps(self, assignment):
        form_key_dict = self._get_form_key_dict(assignment)
        for form1, form2 in combinations(form_key_dict.values(), 2):
            total = sum(count_overlaps(fb1, fb2) for fb1, fb2 in zip(form1, form2))
            if total > self._key_overlaps:
                return False
        return True
    
    
    def _check_usage(self, assignment):
        used_vars = defaultdict(Counter)
        for var, val in assignment.items():
            used_vars[var[1]][val] += 1
            if used_vars[var[1]][val] > self._max_reuse:
                return False
        
        for block, counts in used_vars.items():
            sum(counts.items()) < self._nforms & len(counts)
            
        
            
            
            
        
            
        

SyntaxError: invalid syntax (<ipython-input-61-8f66443d7e97>, line 36)

In [1]:
from collections import Counter
Counter.

In [77]:
from operator import itemgetter
from itertools import groupby
from string import ascii_uppercase
import random

def get_random_keys(n_keys, keys=ascii_uppercase[:4]):
    return [random.choice(keys) for _ in xrange(n_keys)]

def get_random_shuffle(a):
    out = a[:]
    random.shuffle(out)
    return out

def get_block_scram(key, master_dict):
    return get_random_shuffle(master_dict[(key[0], 0)])
n_scrams = 4

master_v = [v[::-1] for v in variables if v[0] == 0]
block_scrams = product(range(1, len(master_v) + 1), range(1, n_scrams + 1))
master_dict = {v : get_random_keys(7) for v in master_v}
block_scrams = {key : get_block_scram(key, master_dict) for key in block_scrams}

master_dict

{(1, 0): ['B', 'B', 'B', 'B', 'B', 'A', 'A'],
 (2, 0): ['D', 'C', 'D', 'B', 'D', 'A', 'C'],
 (3, 0): ['A', 'A', 'A', 'C', 'C', 'D', 'D'],
 (4, 0): ['B', 'D', 'B', 'B', 'A', 'A', 'D'],
 (5, 0): ['B', 'C', 'C', 'A', 'D', 'D', 'C'],
 (6, 0): ['B', 'D', 'B', 'C', 'B', 'B', 'D'],
 (7, 0): ['C', 'C', 'A', 'A', 'D', 'D', 'B'],
 (8, 0): ['C', 'A', 'A', 'B', 'C', 'B', 'C'],
 (9, 0): ['D', 'B', 'D', 'D', 'C', 'D', 'D']}

In [78]:
block_scrams.update(master_dict)
block_scrams

{(1, 0): ['B', 'B', 'B', 'B', 'B', 'A', 'A'],
 (1, 1): ['B', 'B', 'B', 'A', 'A', 'B', 'B'],
 (1, 2): ['B', 'B', 'B', 'A', 'B', 'A', 'B'],
 (1, 3): ['B', 'B', 'A', 'B', 'A', 'B', 'B'],
 (1, 4): ['B', 'A', 'B', 'B', 'B', 'B', 'A'],
 (2, 0): ['D', 'C', 'D', 'B', 'D', 'A', 'C'],
 (2, 1): ['D', 'D', 'A', 'C', 'B', 'D', 'C'],
 (2, 2): ['B', 'D', 'D', 'D', 'C', 'C', 'A'],
 (2, 3): ['D', 'B', 'C', 'C', 'D', 'D', 'A'],
 (2, 4): ['C', 'C', 'D', 'B', 'D', 'A', 'D'],
 (3, 0): ['A', 'A', 'A', 'C', 'C', 'D', 'D'],
 (3, 1): ['A', 'D', 'C', 'A', 'D', 'A', 'C'],
 (3, 2): ['A', 'C', 'C', 'A', 'A', 'D', 'D'],
 (3, 3): ['C', 'C', 'A', 'A', 'D', 'A', 'D'],
 (3, 4): ['D', 'A', 'C', 'C', 'A', 'A', 'D'],
 (4, 0): ['B', 'D', 'B', 'B', 'A', 'A', 'D'],
 (4, 1): ['B', 'A', 'D', 'A', 'D', 'B', 'B'],
 (4, 2): ['A', 'B', 'D', 'A', 'D', 'B', 'B'],
 (4, 3): ['D', 'B', 'A', 'B', 'B', 'A', 'D'],
 (4, 4): ['D', 'B', 'B', 'A', 'D', 'B', 'A'],
 (5, 0): ['B', 'C', 'C', 'A', 'D', 'D', 'C'],
 (5, 1): ['C', 'C', 'A', 'D', 'C',

In [80]:
from collections import Counter
c = Counter()

In [82]:
c.

TypeError: 'Counter' object is not callable

In [73]:
from itertools import islice
def get_name():
    names = NAMES['house']
    return names

NAMES = {'house' : 'garage'}


def change_var():
    NAMES['house'] = random.choice(('house', 'garage', 'garden', 'villa'))
    

def test():
    while True:
        yield get_name()
        change_var()
    

for t in islice(test(), 10):
    print t


garage
house
garden
garage
villa
garage
villa
garden
house
garden


In [121]:
total = 35 + 39 + 9 + 15

In [133]:
points = 33.81 + 34.04 + 9 + 15

In [134]:
points / total

0.9372448979591836

In [132]:
39 * (.79 + .95) / 2.

33.93

In [130]:
(.79 + .95) / 2

0.87

In [131]:
.87 * 35

30.45